In [1]:
import numpy as np
import torch
import pickle
import time
import os
%matplotlib inline
import matplotlib.pyplot as plt
# import gnn

In [2]:
if not os.path.isfile('superpixels.zip'):
    print('downloading..')
    !curl https://www.dropbox.com/s/y2qwa77a0fxem47/superpixels.zip?dl=1 -o superpixels.zip -J -L -k
    !unzip superpixels.zip -d ../
    # !tar -xvf superpixels.zip -C ../
else:
    print('File already downloaded')

File already downloaded


In [3]:

import os
os.chdir('../../') # go to root folder of the project
print(os.getcwd())

/Users/ziangeng/Desktop/Research


In [2]:
import pickle

%load_ext autoreload
%autoreload 2

from superpixels import SuperPixDatasetDGL 

from data import LoadData
from torch.utils.data import DataLoader
from superpixels import SuperPixDataset


In [5]:
# start = time.time()

# DATASET_NAME = 'MNIST'
# trainset=[0,1,2,3,4,5]
# testset=[6,7,8,9]
# way=4
# shot=8
# task=2
# it=0
# datasetlist=[]
# ftlist=[]
# for it in range (task):
#     dataset = SuperPixDatasetDGL(DATASET_NAME,trainset,trainset,it,way,task,shot) 
#     with open('data/superpixels/MNIST.pkl','wb') as f:
#         pickle.dump([dataset.train,dataset.val,dataset.test],f)
#     dataset = LoadData(DATASET_NAME) # 54s
#     print('train',dataset.train.graph_labels)
#     datasetlist.append(dataset)
# for it in range (task):
#     dataset = SuperPixDatasetDGL(DATASET_NAME,testset,testset,it,way,task,shot) 
#     with open('data/superpixels/MNIST.pkl','wb') as f:
#         pickle.dump([dataset.train,dataset.val,dataset.test],f)
#     dataset = LoadData(DATASET_NAME) # 54s
#     print('test',dataset.train.graph_labels)
#     ftlist.append(dataset)



Adj matrix defined from super-pixel locations (only...)
test [1, 3, 0, 4]


FileNotFoundError: [Errno 2] No such file or directory: './data/superpixels/mnist_75sp_test.pkl'

In [3]:
from torch_geometric.datasets import MNISTSuperpixels
import random
from torch.utils.data import random_split,Subset

trainset=[0,1,2,3,4,5]
testset=[6,7,8,9]
# trainset=[0,1,2,3,4,5,6,7,8,9]
# testset=[0,1,2,3,4,5,6,7,8,9]
way=4
shot=10
task=40
SAG = MNISTSuperpixels('data/superpixels/')
# Creates a list containing 5 lists, each of 8 items, all set to 0
indexlist = [[] for y in range(10)] 
# if(SAG[0].y==5):
#     print('hello')
for ind in range (len(SAG)):
    for sa in range(10):
        if(SAG[ind].y==sa):
            indexlist[sa].append(ind)


SAGtrainlist=[]
SAGftlist=[]

for n in range(task):
    train_choice=random.sample(trainset,way)
    ft_choice=random.sample(testset,way)
    tr_indlist=[]
    te_indlist=[]
    ft_indlist=[]
    ftte_indlist=[]
    for t in train_choice:
        tr_indlist=tr_indlist+indexlist[t][n*shot:(n+1)*shot]
        te_indlist=te_indlist+indexlist[t][(len(indexlist[t])-(n+1)*shot):(len(indexlist[t])-n*shot)]
    for f in ft_choice:
#         ft_indlist=ft_indlist+indexlist[f][n*shot:(n+1)*shot]
#         ftte_indlist= ftte_indlist+indexlist[f][(len(indexlist[f])-(n+1)*shot):(len(indexlist[f])-n*shot)]
        ft_indlist=ft_indlist+indexlist[f][(task+n)*shot:(task+n+1)*shot]
        ftte_indlist= ftte_indlist+indexlist[f][(len(indexlist[f])-(task+n+1)*shot):(len(indexlist[f])-(task+n)*shot)]
    sup_qur_tr=[]
    sup_qur_ft=[]
    sup_qur_tr.append(Subset(SAG,tr_indlist))
    sup_qur_tr.append(Subset(SAG,te_indlist))
    sup_qur_ft.append(Subset(SAG,ft_indlist))
    sup_qur_ft.append(Subset(SAG,ftte_indlist))
    SAGtrainlist.append(sup_qur_tr)
    SAGftlist.append(sup_qur_ft)

print('finish')   
    


finish


In [4]:
# from SGAPoolORI import SAGLearnerori

# testori=SAGLearnerori(SAG)
# out,pred,loss,correct=testori()
# print(len(SAGftlist[0][1]))
# for i in range (len(SAGftlist[0][0])):
#     print(SAGftlist[0][1][i].y)



# from HGPORILearner import HGPORILearner

# testoriHGP=HGPORILearner(SAG)
# acc,loss,correct=testoriHGP()
# print(acc)
# print(loss)

In [ ]:
# import torch


# from SAGLearner import SAGLearner

# test=SAGLearner(4)



# print(type(test.dataset[0]))
# print(type(test.dataset))
# print('first',test.dataset[0].y)
# print('second',test.dataset[1].y)
# print(test.dataset.num_classes)
# print(test.dataset.num_features)



# pred,out,loss,correct=test(SAGtrainlist[0],'test')
# print('pe',pred)
# print('out',correct)
# # print()
# # test('train')
# for g in range(len(test.modelbn)):
#     print('bn',g,test.modelbn[g])
#     print('para',g,list(test.model.parameters())[g])


In [ ]:
from HGPmeta import HGPMeta
from    HGPLearner import HGPLearner
from    copy import deepcopy
# hgtest=HGPLearner(4)
# net=deepcopy(hgtest)  
# print('done')
metalist=[0.0001,0.0002,0.0003,0.0005,0.0007,0.001,0.002,0.003,0.005,0.007,0.01,0.02,0.03,0.05]

result=[]
# m=metalist[3]
metalist=[0.001]
m=0.001
acclist=[]
for s in metalist:
        
    hgtest=HGPMeta(s,m,way)
    acc,loss=hgtest(SAGtrainlist,SAGtrainlist)
    acctestlist=[]
    acclist.append([s,m,acc,loss])
    for i in range (task):
    
        acctest=hgtest.finetunning(SAGftlist[i],SAGftlist[i])
        acctestlist.append(acctest)
    maccs = np.array(acctestlist).mean(axis=0).astype(np.float16)
    result.append([s,m,maccs])
    
for d in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' train set step: ',acclist[d][0],' meta: ',acclist[d][1],' acc: ',acclist[d][2],' loss ',acclist[d][3])

for r in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' step: ',result[r][0],' meta: ',result[r][1],' acc: ',result[r][2])
   

parameters 0.001 0.001
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 0, 2, 3, 1, 3, 1, 1, 3, 3, 0, 1, 2, 0, 3, 2, 0, 2, 2, 0, 1, 0, 2,
        2, 0, 2, 3, 1, 2, 3, 1, 2, 1, 1, 3, 0, 0, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 2, 2, 0, 2, 3, 0, 3, 3, 0, 0, 3, 1, 0, 2, 1, 2, 1, 2, 0, 2, 1, 1,
        3, 3, 0, 1, 2, 1, 0, 3, 0, 3, 3, 2, 1, 3, 2, 1])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 1, 2, 0, 0, 1, 0, 3, 2, 2, 1, 2, 1, 3, 0, 0, 3, 1, 2, 1, 1, 2, 0,
        0, 3, 1, 2, 1, 2, 3, 2, 2, 0, 3, 3, 0, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 1, 2, 2, 1, 0, 0, 3, 3, 3, 1, 2, 3, 1, 1, 2, 2, 0, 1, 0, 3, 0, 0,
        3, 0, 2, 0, 3, 3, 2, 1, 3, 1, 2, 3, 1, 2, 2, 1])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 0, 2, 1, 3, 2, 3, 3, 1, 3, 0, 3, 0, 1, 1, 2, 3, 0, 1, 3, 2, 1, 1,
        2, 3, 1, 2, 3, 1, 2, 0, 2, 0, 3, 2, 0, 0, 2, 0])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 1, 0, 0, 2, 0, 3, 3, 3, 2, 2, 2, 1, 3, 3, 2, 3, 1, 0, 1, 2, 1, 0,
        0, 2, 1, 2, 2, 3, 2, 3, 3, 1, 0, 1, 0, 3, 1, 0])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 2, 2, 0, 3, 3, 2, 2, 0, 3, 3, 1, 3, 2, 3, 1, 1, 2, 0, 2, 2, 1, 1,
        3, 0, 2, 0, 1, 2, 0, 0, 0, 3, 0, 1, 1, 3, 1, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0,

...

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 2, 3, 0, 0, 2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 0, 1, 3, 2, 3, 0, 1,
        0, 3, 3, 3, 3, 2, 2, 2, 3, 0, 3, 2, 3, 0, 1, 1])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 1, 2, 3, 1, 0, 0, 2, 3, 2, 3, 0, 3, 3, 1, 0, 0, 0, 0, 1, 3, 3, 3,
        1, 2, 2, 1, 0, 1, 2, 3, 3, 2, 2, 0, 2, 1, 1, 2])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 0, 1, 2, 2, 3, 3, 0, 3, 2, 3, 2, 1, 0, 3, 1, 2, 0, 2, 3, 1, 2, 3,
        0, 3, 1, 1, 0, 0, 2, 1, 1, 0, 3, 2, 0, 1, 3, 2])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 1, 0, 1, 2, 3, 2, 3, 1, 1, 2, 1, 2, 3, 3, 1, 3, 0, 3, 1, 2, 0, 2,
        3, 2, 3, 2, 0, 1, 0, 3, 0, 1, 2, 0, 0, 1, 2, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 1, 2, 2, 1, 0, 3, 0, 2, 3, 2, 0, 3, 1, 2, 3, 3, 1, 0, 3, 3, 0, 1,
        1, 2, 3, 3, 1, 1, 1, 0, 2, 3, 2, 0, 1, 0, 2, 2])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 2, 0, 2, 2, 0, 1, 0, 1, 3, 2, 0, 1, 0, 3, 3, 3, 0, 3, 1, 2, 2, 2,
        1, 3, 0, 0, 2, 1, 0, 3, 1, 2, 3, 3, 2, 1, 1, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0,

pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1, 1, 2, 1, 3, 3, 1, 2, 2, 0, 3, 1, 2, 3, 2, 0, 0, 1, 3, 3, 2, 2, 2,
        0, 2, 0, 1, 1, 3, 0, 0, 1, 0, 3, 3, 0, 1, 3, 2])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 1,

In [2]:
from SGAmeta import SGAMeta


    
metalist=[0.0001,0.0002,0.0003,0.0005,0.0007,0.001,0.002,0.003,0.005,0.007,0.01,0.02,0.03,0.05]

result=[]
# m=metalist[3]
metalist=[5]
m=0.0005
acclist=[]
for s in metalist:
        
    sgtest=SGAMeta(s,m,way)
    acc,loss=sgtest(SAGtrainlist,SAGtrainlist)
    acctestlist=[]
    acclist.append([s,m,acc,loss])
    for i in range (task):
    
        acctest=sgtest.finetunning(SAGftlist[i],SAGftlist[i])
        acctestlist.append(acctest)
    maccs = np.array(acctestlist).mean(axis=0).astype(np.float16)
    result.append([s,m,maccs])
    
for d in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' train set step: ',acclist[d][0],' meta: ',acclist[d][1],' acc: ',acclist[d][2],' loss ',acclist[d][3])

for r in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' step: ',result[r][0],' meta: ',result[r][1],' acc: ',result[r][2])
    
  

parameters 5 0.0005
tq333333
pred tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
z tensor([0, 1, 1, 1, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 2, 2, 0, 3, 3, 3, 1, 3, 3, 3,
        3, 0, 0, 3, 3, 0, 2, 3, 2, 1, 2, 2, 1, 1, 2, 2])
pred tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3,
        3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([0, 0, 0, 3, 1, 0, 1, 0, 0, 0, 3, 3, 3, 0, 3, 0, 1, 0, 3, 0, 3, 3, 3, 0,
        3, 3, 3, 0, 1, 3, 0, 3, 3, 0, 1, 0, 0, 0, 0, 1])
z tensor([0, 1, 2, 0, 2, 1, 3, 0, 0, 2, 0, 3, 1, 2, 3, 3, 0, 0, 3, 1, 2, 3, 1, 2,
        3, 2, 1, 3, 1, 3, 0, 1, 0, 3, 1, 2, 0, 2, 1, 2])
pred tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
z tensor([0, 0,

beforepara Parameter containing:
tensor([[-2.8183e-02, -1.4335e-01, -1.8767e-01, -1.5593e-01, -1.3786e-01,
          3.0450e-02,  2.1795e-01, -1.3935e-01,  2.2110e-01,  2.5892e-02,
          2.0227e-01, -1.6653e-01, -7.9004e-02,  2.8790e-02, -2.1883e-01,
          1.9499e-01, -8.0286e-03, -3.3159e-02, -1.6076e-01, -2.0088e-01,
          2.8072e-02,  7.3338e-02,  3.9233e-02, -7.1420e-02, -7.6887e-02,
          9.4579e-02, -2.3163e-01, -1.5090e-01, -2.0007e-01,  1.0080e-01,
         -1.3066e-01, -2.2702e-01, -4.8934e-04,  1.9134e-01, -1.8364e-01,
         -4.8692e-02, -1.5168e-01,  1.0094e-01,  2.0614e-01,  1.8895e-01,
         -1.6186e-01,  2.0617e-01, -2.2670e-01,  1.4405e-02,  2.1052e-01,
          9.9725e-02, -5.3885e-02,  1.2192e-01, -1.8397e-01, -8.7953e-02,
          1.1133e-01,  7.6022e-02, -1.5607e-02,  1.6136e-01, -4.5111e-02,
         -1.5703e-01, -4.5178e-02, -1.0896e-01, -1.7389e-01, -8.0025e-02,
          1.2370e-01, -1.8045e-02,  1.4158e-01, -1.1497e-01,  1.6906e-01,
     

pred tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
z tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
pred tensor([2, 1, 3, 1, 3, 3, 2, 2, 2, 2, 0, 2, 1, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 2, 2, 1, 1, 2, 2, 3, 2, 1, 2, 0, 2, 0, 1])
z tensor([0, 0, 1, 0, 2, 2, 2, 0, 3, 3, 1, 0, 2, 1, 0, 3, 1, 0, 0, 3, 1, 2, 2, 2,
        3, 2, 3, 1, 0, 1, 2, 3, 3, 0, 2, 1, 1, 3, 1, 3])
fw ft 2 tensor([[-2.7685e-02, -1.4285e-01, -1.8717e-01, -1.5543e-01, -1.3736e-01,
          2.9974e-02,  2.1751e-01, -1.3885e-01,  2.2075e-01,  2.5299e-02,
          2.0197e-01, -1.6603e-01, -7.8504e-02,  2.8372e-02, -2.1833e-01,
          1.9439e-01, -7.5347e-03, -3.2661e-02, -1.6026e-01, -2.0038e-01,
          2.7487e-02,  7.3028e-02,  3.8702e-02, -7.0920e-02, -7.6388e-02,
          9.4072e-02, -2.3113e-01, -1.5040e-01, -1.9957e-01,

In [ ]:
from meta import Meta
support=[]
query=[]
supporty=[]
queryy=[]

for it in range (20):
    support.append(datasetlist[it])
    query.append(datasetlist[it])

    supporty.append(datasetlist[it].train.graph_labels)

    queryy.append(datasetlist[it].test.graph_labels)

    
metalist=[0.0001,0.0002,0.0005,0.0007,0.001,0.002,0.005,0.007,0.01,0.0003,0.003,0.006]
# metalist=[0.0001,0.0005,0.0007,0.001,0.005,0.007,0.01]
# metalist=[0.0007]
result=[]
# m=metalist[11]
m=0.0007
acclist=[]
for s in metalist:
        
    test=Meta(s,m,way)
    acc,loss=test(support,supporty,query,queryy)
    acctestlist=[]
    acclist.append([s,m,acc,loss])
    for i in range (task):
    
        acctest=test.finetunning(ftlist[i], ftlist[i].train.graph_labels, ftlist[i], ftlist[i].test.graph_labels)
        acctestlist.append(acctest)
    maccs = np.array(acctestlist).mean(axis=0).astype(np.float16)
    result.append([s,m,maccs])
    
for d in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' train set step: ',acclist[d][0],' meta: ',acclist[d][1],' acc: ',acclist[d][2])

for r in range (len(result)):
    
    print(' way: ',way,' shot: ',shot,' step: ',result[r][0],' meta: ',result[r][1],' acc: ',result[r][2])
    
    

In [ ]:
#### import tensorflow as tf
x_train_tensor=torch.FloatTensor([0.01])
# testensor=torch.nn.Parameter(testensor)
# # testensor=testensor+testensor
# print(type(testensor))
a = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
a.retain_grad()
b = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
c = a + 1
# c.data = c.data + 1
# print(c)
# print('aa',a)
c.retain_grad()
d = torch.nn.Parameter(c, requires_grad=True,)
for epoch in range(2):
    yhat = d + b * x_train_tensor
    error = x_train_tensor - yhat
    loss = (error ** 2).mean()
    loss.backward()
    print('a',a.grad)
    print('b',b.grad)
    print('c',c.grad)
    print('d',d.grad)

In [ ]:
grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook
a = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
b = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
c = a + 1
# c.register_hook(save_grad(0))
# a.register_hook(save_grad(1))
print('cg1',grads)
# c.retain_grad()
# grad = torch.autograd.grad(loss, c,allow_unused=True)
d = c
# d = torch.nn.Parameter(c, requires_grad=True,)
for epoch in range(2):
    yhat = d + b * x_train_tensor
    error = x_train_tensor - yhat
    loss = (error ** 2).mean()
    grad = torch.autograd.grad(loss, c,allow_unused=True)
#     loss.backward()
    print('grad',grad)
    print('a',a.grad)
    print('b',b.grad)
    print('c',c.grad)
    print('d',d.grad)
    print('cg2',grads)

In [ ]:
from    torch.nn import functional as F
from    torch import nn
w = torch.randn(8,4,3,3)
x= tf.constant([1,2,0,1,2])
py= tf.constant([[0.1,0.4,0.5],[0.1,0.4,0.5],[0.1,0.4,0.5],[0.1,0.4,0.5],[0.1,0.4,0.5]])
print(x)
# vars = nn.ParameterList()
# vars.append(w)

# b = torch.randn(8,4,3,3)
lm= nn.Linear(4, 6)
y=lm(x)
# y.backward()
criterion = nn.CrossEntropyLoss()
print(y,py)
loss = criterion(py,y)


# print(y)
# print(x.grad)
# F.conv2d(inputs, filters, padding=1)
# y=torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)                
# y = F.conv2d(x, w,padding=1)
# y.backward()
# loss_q = F.cross_entropy(y, y)


In [ ]:
# import torch
# import torch.nn as nn
# from torch.optim import Adam

# class NN_Network(nn.Module):
#     def __init__(self,in_dim,hid,out_dim):
#         super(NN_Network, self).__init__()
#         self.linear1 = nn.Linear(in_dim,hid)
#         self.linear2 = nn.Linear(hid,out_dim)
#         self.linear1.weight = torch.nn.Parameter(torch.zeros(in_dim,hid))
#         self.linear1.bias = torch.nn.Parameter(torch.zeros(hid))
#         self.linear2.weight = torch.nn.Parameter(torch.zeros(in_dim,hid))
#         self.linear2.bias = torch.nn.Parameter(torch.ones(hid))
#         print('p1',self.linear1.weight)
#         print('p1',self.linear1.bias)
#         print('p3',self.linear2.weight)
#         print('p1',self.linear2.bias)
#     def forward(self, input_array):
#         h = self.linear1(input_array)
#         y_pred = self.linear2(h)
#         return y_pred

# in_d = 5
# hidn = 2
# out_d = 3
# net = NN_Network(in_d, hidn, out_d)
# print('check',list(net.parameters()))
# for param in net.parameters():
#     print(type(param.data), param.size())

In [ ]:
from    learner import Learner
test=Learner(4)
ori=test.parameters()
# print('onigi',test.vars[1].weight)
loss1,pred1,acc1=test(datasetlist[0],'train',init=True)
loss2,pred2,acc2=test(datasetlist[0],'test',init=True)


# test(datasetlist[1],'train')
# test(datasetlist[2],'train')
# print('aft',aft.weight)
# print('onigi',test.vars[1].weight)
print('acc1',acc1)
print('testloss',loss1)
print('labelout1',datasetlist[0].train.graph_labels)
print('pred',pred1)
print('acc2',acc2)
# print('score',escore)
# print('label',elabel)

print('testloss2',loss2)
# print('score',escore)
# print('label',elabel)
print('labelout2',datasetlist[0].test.graph_labels)
print('pred2',pred2)
# print(test.parameters())
# print(type(elabel))
# print(type(pred))

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
y = 0.01*x + 2
print(y)
# z = y * y * 3
# out = z.mean()
# f=out-2
y.backward()
print(x.grad)

In [ ]:
from    torch import nn
n = 70 # num of points

# x is a tensor
x = torch.linspace(0, 10, steps=n)
k = torch.tensor(2.5)

# y is a tensor
y = k*x + 5*torch.rand(n)

# loss function
def mse(y_hat, y): return ((y_hat-y)**2).mean()

a = torch.tensor(-1., requires_grad=True)
a = nn.Parameter(a)
print('ori',a)
y_hat = a*x

loss = mse(y_hat, y) 
# print(loss) 
loss.backward(a)
# print(a.grad)
a.grad.zero_()
print('new',a)
lr = 0.1
a = a - lr * a.grad

In [ ]:
# def plot_histo_graphs(dataset, title):
#     # histogram of graph sizes
#     graph_sizes = []
#     for graph in dataset:
#         graph_sizes.append(graph[0].number_of_nodes())
#         #graph_sizes.append(graph[0].number_of_edges())
#     plt.figure(1)
#     plt.hist(graph_sizes, bins=20)
#     plt.title(title)
#     plt.show()
#     graph_sizes = torch.Tensor(graph_sizes)
#     print('nb/min/max :',len(graph_sizes),graph_sizes.min().long().item(),graph_sizes.max().long().item())
    
# plot_histo_graphs(dataset.train,'trainset')
# plot_histo_graphs(dataset.val,'valset')
# plot_histo_graphs(dataset.test,'testset')

In [ ]:
# print(len(dataset.train))
# print(len(dataset.val))
# print(len(dataset.test))

# print(dataset.train[0])
# print(dataset.val[0])
# print(dataset.test[0])

In [ ]:
start = time.time()

with open('data/superpixels/MNIST.pkl','wb') as f:
        pickle.dump([dataset.train,dataset.val,dataset.test],f)
        
print('Time (sec):',time.time() - start) # 38s

In [ ]:
DATASET_NAME = 'MNIST'
dataset = LoadData(DATASET_NAME) # 54s
trainset, valset, testset = dataset.train, dataset.val, dataset.test
print(dataset.train.graph_labels)

In [ ]:
from    learner import Learner
test=Learner()

loss=test(dataset)
print('loss',loss)
# from sklearn import preprocessing 

# label_encoder = preprocessing.LabelEncoder() 

# check = [4,7,7,6,4,4,7,6,7,4,4,4]
# trans=torch.tensor(label_encoder.fit(range(10)))
# trans=label_encoder.fit([4,6,7])
# change=trans.transform(check)
# print(check)
# print(change)

In [ ]:
# # print(test_acc)
# # print(train_acc)
from    torch.nn import functional as F
input = torch.randn(15, 4, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 3,1,1,3,1,3,1,1,3,1,3,1,3])
output = F.nll_loss(F.log_softmax(input), target)
output.backward()
print(input)
print(target)
print(output)


In [ ]:
start = time.time()
input = torch.randn(20, 4, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([4,4,0,4,1,4,0,1,4,0,4,1,4,0,4,1,4,0,4,1])
batch_size = 10
collate = dataset.collate
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=collate)
# for iter, (batch_graphs, batch_labels, batch_snorm_n, batch_snorm_e) in enumerate(train_loader):
#     print(batch_labels)
print('Time (sec):',time.time() - start) # 0.0003s

#### 